In [200]:
%pip install langchain-core langchain_openai langgraph python-dotenv

import getpass
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

In [201]:
# Load environment variables from .env file
load_dotenv()

# Initialize the model
model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model_name=os.environ["AZURE_OPENAI_MODEL_NAME"],
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),    
)

In [202]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [203]:
model.invoke([HumanMessage(content="What's my name?")])

In [204]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

In [205]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory and create app
app = workflow.compile(checkpointer=MemorySaver())

In [206]:
config = {"configurable": {"thread_id": "1"}}

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [207]:
config = {"configurable": {"thread_id": "1"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [208]:
config = {"configurable": {"thread_id": "2"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [209]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [210]:
async def call_model_with_prompt(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}

workflow_pirate = StateGraph(state_schema=MessagesState)
workflow_pirate.add_edge(START, "model")
workflow_pirate.add_node("model", call_model_with_prompt)
app_pirate = workflow_pirate.compile(checkpointer=MemorySaver())

In [211]:
config = {"configurable": {"thread_id": "3"}}

query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = await app_pirate.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [212]:
config = {"configurable": {"thread_id": "3"}}

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = await app_pirate.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [213]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [214]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

async def call_model_with_parameter(state: State):
    chain = prompt | model
    response = await chain.ainvoke(state)
    return {"messages": [response]}

workflow_with_parameter = StateGraph(state_schema=State)
workflow_with_parameter.add_edge(START, "model")
workflow_with_parameter.add_node("model", call_model_with_parameter)
app_with_parameter = workflow_with_parameter.compile(checkpointer=MemorySaver())

In [215]:
config = {"configurable": {"thread_id": "4"}}

query = "Hi! I'm Bob."

language = "Spanish"

input_messages = [HumanMessage(query)]
output = await app_with_parameter.ainvoke(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
)
output["messages"][-1].pretty_print()

In [216]:
config = {"configurable": {"thread_id": "4"}}

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = await app_with_parameter.ainvoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

In [217]:
from langchain_core.messages import SystemMessage, trim_messages, ToolMessage

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
    end_on=("human", "tool"),
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

In [218]:
async def call_model_with_prompt_and_history(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = await chain.ainvoke(
        {
            "messages": trimmed_messages, 
            "language": state["language"]
        }
    )
    return {"messages": [response]}

workflow_with_prompt_and_history = StateGraph(state_schema=State)
workflow_with_prompt_and_history.add_edge(START, "model")
workflow_with_prompt_and_history.add_node("model", call_model_with_prompt_and_history)
app_with_prompt_and_history = workflow_with_prompt_and_history.compile(checkpointer=MemorySaver())

In [219]:
config = {"configurable": {"thread_id": "5"}}

query = "What is my name?"

language = "English"

input_messages = messages + [HumanMessage(query)]
output = await app_with_prompt_and_history.ainvoke(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
)
output["messages"][-1].pretty_print()

In [220]:
config = {"configurable": {"thread_id": "6"}}

query = "What math problem did I ask?"

language = "English"

input_messages = messages + [HumanMessage(query)]
output = await app_with_prompt_and_history.ainvoke(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
)
output["messages"][-1].pretty_print()

In [221]:
def call_model_with_prompt_and_history_stream(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {
            "messages": trimmed_messages, 
            "language": state["language"]
        }
    )
    return {"messages": [response]}

workflow_with_prompt_and_history_stream = StateGraph(state_schema=State)
workflow_with_prompt_and_history_stream.add_edge(START, "model")
workflow_with_prompt_and_history_stream.add_node("model", call_model_with_prompt_and_history_stream)
app_with_prompt_and_history_stream = workflow_with_prompt_and_history_stream.compile(checkpointer=MemorySaver())


config = {"configurable": {"thread_id": "7"}}

query = "Hi I'm Todd, please tell me a joke."

language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app_with_prompt_and_history_stream.stream(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")